# BDF Introduction

The Jupyter notebook for this demo can be found in:
   - docs/quick_start/demo/bdf_demo.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/bdf_demo.ipynb

Import pyNastran

In [1]:
from IPython.core.display import HTML
import os
import pyNastran
print (pyNastran.__file__)
print (pyNastran.__version__)
pkg_path = pyNastran.__path__[0]
pkg_path = r'C:\Users\sdoyle\Desktop\demo\pyNastran\pyNastran'

from pyNastran.bdf.bdf import BDF, read_bdf
from pyNastran.utils import object_attributes, object_methods

print("pkg_path = %s" % pkg_path)
# TODO: broken for cpylog 1.5.0 beause of new logging approach

c:\nasa\m4\formats\git\pynastran\pyNastran\__init__.py
1.4.0+dev.eded29372
pkg_path = C:\Users\sdoyle\Desktop\demo\pyNastran\pyNastran


## Loading a BDF
There are two ways to load a BDF; the long way or the short way.

The short way instantiates the **```BDF```** class and the short way uses the **```read_bdf```** function.
As this demo was written for the Jupyter Notebook, we'll use **``read_bdf``** and then mention the other method.  The class-based method allows finer control over things like:
 - what cards should be loaded
 - OpenMDAO dynamic syntax support

### The class-based method

In [2]:
bdf_filename = os.path.abspath(os.path.join(pkg_path, '..', 'models', 'iSat', 'ISat_Launch_Sm_Rgd.dat'))
print(bdf_filename)

# create the BDF object
bdf = BDF()

# read the file from the GUI
# don't cross-reference
bdf.read_bdf(bdf_filename, xref=False)

C:\Users\sdoyle\Desktop\demo\pyNastran\models\iSat\ISat_Launch_Sm_Rgd.dat


### The function-based method

In [ ]:
bdf = read_bdf(bdf_filename, xref=False)

For simplicity of using the demo, we'll again use the ```read_bdf``` method

In [3]:
#bdf_filename = r'D:\work\pynastran_0.8.0_py27\models\iSat\ISat_Launch_Sm_Rgd.dat'
bdf_filename = os.path.abspath(os.path.join(pkg_path, '..', 'models', 'iSat', 'ISat_Launch_Sm_Rgd.dat'))

# read the file as a path
bdf_xref = read_bdf(bdf_filename, xref=True)

We can use the generic object attributes/methods functions

#### Some other very handy methods that will be used later by ```test_bdf```

In [4]:
print(bdf.get_bdf_stats())
print("card_count = %s\n" % bdf.card_count)
print("reject_count = %s" % bdf.reject_count)

---BDF Statistics---
SOL 103

bdf.spcs[1]: 1
  SPC:     1

bdf.params: 8
  PARAM    : 8

bdf.nodes: 5380
  GRID     : 5380

bdf.coords: 75
  CORD2R   : 75

bdf.elements: 5568
  CBAR     : 827
  CBUSH    : 104
  CHEXA    : 25
  CQUAD4   : 4580
  CTRIA3   : 32

bdf.rigid_elements: 44
  RBE2     : 44

bdf.properties: 33
  PBAR     : 1
  PBARL    : 18
  PBUSH    : 2
  PSHELL   : 8
  PSOLID   : 4

bdf.masses: 15
  CONM2    : 15

bdf.materials: 22
  MAT1     : 14
  MAT8     : 8

bdf.methods: 1
  EIGRL    : 1

bdf.usets: 1
  USET     : 1


card_count = {'ENDDATA': 1, 'PARAM': 8, 'SPC': 1, 'USET': 1, 'EIGRL': 1, 'CORD2R': 75, 'GRID': 5380, 'CQUAD4': 4580, 'CBAR': 827, 'CHEXA': 25, 'RBE2': 44, 'CTRIA3': 32, 'CBUSH': 104, 'CONM2': 15, 'MAT1': 14, 'MAT8': 8, 'PSHELL': 8, 'PBARL': 18, 'PSOLID': 4, 'PBAR': 1, 'PBUSH': 2}

reject_count = {}


## Cross-referencing

Cross-referencing a BDF allows improved usability of the **``BDF``** class.  It comes with some negative side effects, but in general is a very useful thing.  It dramatically minimizes the amount of code you need to write, greatly simplifies future operations, and is highly recommended.

The major downside is it slows down the code.

### Without Cross-Referencing (xref=False)
Here the raw values of the the data objects are returned to us

In [5]:
cquad = bdf.elements[1]
print(cquad.rstrip())
print(f"{'name':<8}{'eid':>8}{'pid':>8}{'n1':>8}{'n2':>8}{'n3':>8}{'n4':>8}\n")

$*
$*  ELEMENT CARDS
$*
CQUAD4         1       1       1       2       4       3
name         eid     pid      n1      n2      n3      n4



In [7]:
nid1 = cquad.nodes[0]
print("nid1 = %s" % nid1)
n1 = bdf.nodes[nid1]
print(n1)

nid1 = 1
$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4



In [10]:

cd4 = n1.cd
c4 = bdf.coords[cd4]
print(c4)
print("i (xref=False) = %s" % str(c4.i))
print(object_attributes(c4))

CORD2R         4              0.      0.      0.      0.      0.      1.
              1.      0.      0.

i (xref=False) = [1. 0. 0.]
['Type', 'cid', 'comment', 'e1', 'e2', 'e3', 'global_to_local', 'i', 'is_resolved', 'j', 'k', 'local_to_global', 'origin', 'rid', 'rid_ref', 'rid_trace', 'type']


### Cross-Referenced (xref=True)
Here we can trace the referenced objects very easily.

A cross-referenced attribute is indicated with the **``*_ref``** suffix:
  * ``cquad4_element.nodes`` : not cross referenced
  * ``cquad4_element.nodes_ref`` : cross referenced

In [11]:
print("i (xref=True) = %s" % bdf_xref.elements[1].nodes_ref[0].cd_ref.i)

i (xref=True) = [1. 0. 0.]


So how is this done?

In [14]:
print(n1)
print('-----------------')
cquad.nodes_ref = []
cquad.nodes_ref.append(n1)
print(cquad.nodes_ref[0].rstrip())
print(f"{'name':<8}{'nid':>8}{'cp':>8}{'x':>8}{'y':>8}{'z':>8}{'cd':>8}\n")


$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4

-----------------
$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4
name         nid      cp       x       y       z      cd



#### Let's show off the GRID card

In [17]:
# some Grid methods
n1 = bdf_xref.nodes[1]
print(n1.rstrip())
print(f"{'name':<8}{'nid':>8}{'cp':>8}{'x':>8}{'y':>8}{'z':>8}{'cd':>8}\n")

# the comment
c1 = bdf_xref.nodes[1].comment
c2 = bdf_xref.nodes[2].comment
print("c1=%r" % c1)
print("c2=%r" % c2)


# get the position of a node
# in the local cooordinate system
print("xyz = %s" % n1.xyz)

# in the global frame
print("position = %s" % n1.get_position())

# in an arbitrary frame
print("wrt5 = %s" % n1.get_position_wrt(bdf, 5))
print("wrt4 = %s" % n1.get_position_wrt(bdf, 4))


print(bdf.coords[5])


$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4
name         nid      cp       x       y       z      cd

c1='$*\n$*  GRID CARDS\n$*\n'
c2=''
xyz = [ -4.5  -7.5 -14. ]
position = [ -4.5  -7.5 -14. ]
wrt5 = [  2.12132034  14.         -26.59188309]
wrt4 = [ -4.5  -7.5 -14. ]
CORD2R         5         12.8033 12.8033      0. 25.6066 25.6066      0.
         25.6066-1.78-15      0.



Now let's modify the **``GRID``** card and write it out

In [19]:
n1 = bdf_xref.nodes[1]
n1.xyz[1] = -7.5
print("repr  = %s" % n1.repr_fields())
print("raw   = %s" % n1.raw_fields())

n1.xyz[1] = 100000000000.
print("repr2 = %s" % n1.repr_fields())
print(n1)
print(n1.write_card(size=8))
print(n1.write_card(size=16, is_double=False))
print(n1.write_card(size=16, is_double=True))

repr  = ['GRID', 1, 4, -4.5, -7.5, -14.0, 4, '', None]
raw   = ['GRID', 1, 4, -4.5, -7.5, -14.0, 4, '', 0]
repr2 = ['GRID', 1, 4, -4.5, 100000000000.0, -14.0, 4, '', None]
$*
$*  GRID CARDS
$*
GRID           1       4    -4.5   1.+11    -14.       4

$*
$*  GRID CARDS
$*
GRID           1       4    -4.5   1.+11    -14.       4                

$*
$*  GRID CARDS
$*
GRID*                  1               4            -4.5   100000000000.
*                   -14.               4                                

$*
$*  GRID CARDS
$*
GRID*                  1               4-4.500000000D+001.0000000000D+11
*       -1.400000000D+01               4                                



### Examples of xref on elements

In [20]:
eid100 = bdf_xref.elements[100]
print(f"{'name':<8}{'eid':>8}{'pid':>8}{'n1':>8}{'n2':>8}{'n3':>8}{'n4':>8}")
print(eid100)

name         eid     pid      n1      n2      n3      n4
CQUAD4       100       1     149     152     161     160



In [21]:
print("nodes = %s" % eid100.nodes)
print("--node0--\n%s" % eid100.nodes_ref[0])
print("--cd--\n%s" % eid100.nodes_ref[0].cd)
print("cd.cid = %s" % eid100.nodes_ref[0].cd_ref.cid)

print("area = %s" % eid100.Area())
print("mass = %s" % eid100.Mass())

nodes = [149, 152, 161, 160]
--node0--
GRID         149       4      3.     7.5   -16.5       4

--cd--
4
cd.cid = 4
area = 3.75
mass = 3.642880307396999e-05


In [24]:
print("--pid--\n%s" % eid100.pid)
print("pid.pid = %s" % eid100.pid_ref.pid)
print("pid.Pid() = %s" % eid100.Pid())
print(eid100.pid_ref)

--pid--
1
pid.pid = 1
pid.Pid() = 1
$*
$*  PROPERTY CARDS
$*
$*
$*  I-DEAS property: 1  name: BUS PNL HCMB 2PLY
PSHELL         1       6    .036       61415.815       7  22.234 3.551-6
              .4     -.4



In [27]:
print(eid100.pid_ref.mid1_ref)
print("type = %s" % eid100.pid_ref.mid1_ref.type)
print("nu12 = %s" % eid100.pid_ref.mid1_ref.nu12)
print("mass = %s" % eid100.Mass())
from pyNastran.utils import object_stats
print(object_stats(eid100.pid_ref.mid1_ref))

$*
$*  I-DEAS Material: 6  name: BUS_CFRP_PW_ORTHO
$* M46J PW ETW
MAT8           6   1.7+7   1.7+7     .98 340000. 180000. 180000..0001712
                           71.33

type = MAT8
nu12 = 0.98
mass = 3.642880307396999e-05
MAT8:
  F12    : 0.0
  S      : 0.0
  TRef   : 71.33
  Xc     : 0.0
  Xt     : 0.0
  Yc     : 0.0
  Yt     : 0.0
  a1     : 0.0
  a2     : 0.0
  comment : '$*\n$*  I-DEAS Material: 6  name: BUS_CFRP_PW_ORTHO\n$* M46J PW ETW\n'
  e11    : 17000000.0
  e22    : 17000000.0
  g12    : 340000.0
  g1z    : 180000.0
  g2z    : 180000.0
  ge     : 0.0
  mats8  : None
  matt8  : None
  matt8_ref : None
  mid    : 6
  mp_name_map : {'E1': 'e11', 'E2': 'e22', 'NU12': 'nu12', 'G12': 'g12', 'G1Z': 'g1z', 'RHO': 'rho', 'A1': 'a1', 'A2': 'a2'}
  nu12   : 0.98
  rho    : 0.000171204
  strn   : 0.0
  tref   : 71.33
  type   : 'MAT8'



## Write the modified deck
Let's first switch to the desktop to make the file easy to find

In [28]:
import getpass
name = getpass.getuser()
os.chdir(os.path.join(r'C:\Users', name, 'Desktop'))

In [29]:
pwd

'C:\\Users\\sdoyle\\Desktop'

There are two ways to write a deck
 - **``interspersed``** : alternate properties and elements (similar to how Patran writes decks)
 - **``not-interspersed (default)``** : much faster
 
We can also use 8 or 16 character field width as well as double precision.

Note that double precision only works for certain cards (e.g. ``GRID``, ``COORD``, ``DMIG``) and not much else.

In [30]:
bdf_xref.write_bdf('fem.bdf', interspersed=False, size=8, is_double=False)
!tail -n 5 "fem.bdf"

#bdf_xref.write_bdf('fem_interspersed.bdf', interspersed=True, size=16, is_double=False)
#!tail "fem.bdf"

bdf_xref.write_bdf('fem_inters.bdf', interspersed=True, size=16, is_double=True)
!tail "fem.bdf"

CORD2R        75        1.355-13-2.19-15    -40.1.355-13-2.19-15      0.
             40.-2.19-15    -40.
CORD2R        76        1.355-13-2.19-15    -40.1.355-13-2.19-15      0.
             40.-2.19-15    -40.
ENDDATA


        -51.1378    20.1    -38.
CORD2R        73            -7.5    20.1    -38.    -7.563.73783    -38.
        -51.1378    20.1    -38.
CORD2R        74            -7.5    20.1    -38.    -7.563.73783    -38.
        -51.1378    20.1    -38.
CORD2R        75        1.355-13-2.19-15    -40.1.355-13-2.19-15      0.
             40.-2.19-15    -40.
CORD2R        76        1.355-13-2.19-15    -40.1.355-13-2.19-15      0.
             40.-2.19-15    -40.
ENDDATA


In [ ]:
#bdf_filename